In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

%matplotlib inline
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

HousePrice1 = pd.read_csv("HousePrice1.csv")

HousePrice1.head(5)

In [ ]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="hi")
location = locator.geocode('3416 WEST ST' + ', Ames, IA')

HousePrice1['Geo_Addr'] = [locator.geocode(addr+', Ames, IA') if type(addr)==str else None \
for addr in HousePrice1['Prop_Addr']]


In [ ]:
Geo_Addr